In [19]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
from astropy import units as u
from astropy import constants as c
from poliastro.bodies import Body
from poliastro.twobody import Orbit

In [ ]:
# All of this inital code is getting the user's input on the initial conditions of the system
m_star = float(input("How big should the Star be in kg?")) * u.kg
m_planet = float(input("How big should the Planet be in kg?")) * u.kg
m_total = c.G * (m_star + m_planet)
sys_center = Body(None, m_total, "System Center")
r_input = float(input("How far should the bodies be from each other in AU?")) * u.AU
r_input = r_input.to(u.m)
rx = -r_input/(np.sqrt(2))
ry = r_input/(np.sqrt(2))
rx = rx.to(u.km)
ry = ry.to(u.km)
eccentricity = float(input("How eccentric should the orbit be? Choose a value between 0 and 1, with 0 being circular."))
a = r_input / (1-eccentricity)
v = np.sqrt(m_total * ((2/r_input) - (1/(a))))
vx = v/(np.sqrt(2))
vy = v/(np.sqrt(2))
r0 = u.Quantity([rx, ry, 0 * u.km])
v0 = u.Quantity([vx, vy, 0 * u.km / u.s])
orb = Orbit.from_vectors(sys_center, r0, v0)
resp = input("Show Kepler's Second Law? (True or False): ").strip().lower()
show_Kepler = resp == "true"

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# This function creates the orbital projection lines by decomposing the relative position and velocity into x and y coords for both bodies
def get_orbit_coords(times):
    rx, ry = [], []
    for t in times:
        r_vec = orb.propagate(t).r.to(u.km).value
        rx.append(r_vec[0])
        ry.append(r_vec[1])
    rel_x = np.array(rx)
    rel_y = np.array(ry)
    m_total = m_star + m_planet
    ratio_star = (m_planet / m_total).decompose().value
    ratio_planet = (m_star / m_total).decompose().value
    sx = -ratio_star * rel_x
    sy = -ratio_star * rel_y
    px = +ratio_planet * rel_x
    py = +ratio_planet * rel_y
    return sx, sy, px, py

# This creates the swept arcs which demonstrate Kepler's Second Law if desired
def get_wedge_polygon(start_fraction, duration_fraction, steps=50):
    t_start = start_fraction * orb.period
    t_end = (start_fraction + duration_fraction) * orb.period
    times = np.linspace(t_start.value, t_end.value, steps) * u.s
    rx, ry = [], []
    for t in times:
        r_vec = orb.propagate(t).r.to(u.km).value
        rx.append(r_vec[0])
        ry.append(r_vec[1])
    rel_x, rel_y = np.array(rx), np.array(ry)
    m_total = m_star + m_planet
    ratio_planet = (m_star / m_total).decompose().value
    px = +ratio_planet * rel_x
    py = +ratio_planet * rel_y
    poly_x = np.concatenate([[0], px, [0]])
    poly_y = np.concatenate([[0], py, [0]])
    return poly_x, poly_y

# Asking if the user what portion of the orbit should be swept by the arc to demonstrate Kepler's Second Law
if show_Kepler:
    duration = float(input("Fraction of time of orbit for keplers equal area"))

# Creating the arc dimensions
w1_x, w1_y = get_wedge_polygon(start_fraction=0.0, duration_fraction=duration)
w2_x, w2_y = get_wedge_polygon(start_fraction=0.45, duration_fraction=duration)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# This is all the foundational code for creating the plot and animating it by propagating the orbit through the frames
path_times = np.linspace(0, 1, 200) * orb.period 
star_path_x, star_path_y, planet_path_x, planet_path_y = get_orbit_coords(path_times)
anim_frames = 200
anim_times = np.linspace(0, 1, anim_frames) * orb.period
star_anim_x, star_anim_y, planet_anim_x, planet_anim_y = get_orbit_coords(anim_times)
fig, ax = plt.subplots(figsize=(8, 8))
fig.patch.set_facecolor('black')
ax.set_facecolor('black')
ax.spines['bottom'].set_color('white')
ax.spines['top'].set_color('white') 
ax.spines['right'].set_color('white')
ax.spines['left'].set_color('white')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.yaxis.label.set_color('white')
ax.xaxis.label.set_color('white')
ax.title.set_color('white')
ax.grid(True, color='grey', alpha=0.3)
ax.set_aspect('equal')

# This chooses the scale of the system with a default of about 1.4 AU in each +- xy direction
limit = 2.0e8 
ax.set_xlim(-limit, limit)
ax.set_ylim(-limit, limit)

# This colors the paths of the planet and the star
ax.plot(0, 0, 'wx', label='Center of Mass')
ax.plot(star_path_x, star_path_y, color='orange', linestyle='--', label='Star Path')
ax.plot(planet_path_x, planet_path_y, color='cyan', linestyle='--', label='Planet Path')

# This determines whether to show Kepler's Second Law
if show_Kepler:
    ax.plot(w1_x, w1_y, label = f"Area 1 (t={duration}%)")
    ax.plot(w2_x, w2_y, label = f"Area 2 (t={duration}%)")
    
# Setting up the Legend
legend = ax.legend(loc="upper left", facecolor='black', edgecolor='white')
for text in legend.get_texts():
    text.set_color("white")

# This loops through frames "i" to generate the animation
snapshots = []
for i in range(anim_frames):
    star_dot, = ax.plot(star_anim_x[i], star_anim_y[i], 'o', color='orange', markersize=14)
    planet_dot, = ax.plot(planet_anim_x[i], planet_anim_y[i], 'o', color='cyan', markersize=8)
    connector, = ax.plot([star_anim_x[i], planet_anim_x[i]], 
                         [star_anim_y[i], planet_anim_y[i]], 
                         color = "white", linestyle = "dotted")
    snapshots.append([star_dot, planet_dot, connector])
anim = ArtistAnimation(fig, snapshots, interval=40, blit=True)

# This creates the UI for the animation display
plt.close()
from IPython.display import HTML
HTML(anim.to_jshtml())

How big should the Star be in kg? 2e30
How big should the Planet be in kg? 6e24
How far should the bodies be from each other in AU? 1
